In [6]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer
from transformers import AutoModelWithLMHead
from torch import nn
from transformers import TrainingArguments
from transformers import Trainer
from pathlib import Path
import torch

import re

import tensorflow as tf
from preprocessors import fill_in_the_blank

In [1]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("device: ", device)

device:  cpu


/anaconda/envs/py38_default/lib/python3.8/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272172048/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
%load_ext autoreload
%autoreload 2

In [29]:
version = 'v1'

df = pd.read_pickle(f'/home/student/Desktop/Project/Data/T5Data/PreparedText/prepared_data_{version}.pkl')


In [38]:
labeled_df = pd.read_pickle(f'/home/student/Desktop/Project/Data/T5Data/LabeledData/labeled_data_{version}.pkl')


In [39]:
labeled_df

,input_ids,labels
0,"fill song ['canadian pop', 'pop', 'post-teen p...","10,000 <extra_id_1> 3rd fret<n><t><s2><t><t> <..."
1,"fill song ['canadian pop', 'pop', 'post-teen p...",<extra_id_1> Hours<n><n>Capo on <extra_id_2> 1...
2,"fill song ['canadian pop', 'pop', 'post-teen p...",2 Much<n><n><t><s2><t><t><n>Intro: F#m7<s2>D2 ...
3,"fill song ['canadian pop', 'pop', 'post-teen p...",<extra_id_1> stare too much <n><s2><n>Maybe no...
4,"fill song ['canadian pop', 'pop', 'post-teen p...",2u (feat. David Guetta)<n><n><t><s2><t><t><n>E...
...,...,...
87376,"fill song ['classic rock', 'folk', 'folk rock'...",The Prophet<n><n><t><s2><t><t><n><s3><n><s3><n...
87377,"fill song ['classic rock', 'folk', 'folk rock'...",<extra_id_1> tab<s2>Intro:<s3>G<s3>^<s2>v<s3>G...
87378,"fill song ['classic rock', 'folk', 'folk rock'...",The Race<n><n><t><s2><t><t>Intro:<s2>Cadd9<s2>...
87379,"fill song ['classic rock', 'folk', 'folk rock'...",<extra_id_1> <n>The <extra_id_2> <n>Some will ...


In [67]:
class FillingModel(nn.Module):

    def __init__(self):
        super(FillingModel, self).__init__()
        self.model = AutoModelWithLMHead.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def forward(self, input_ids, attention_mask, labels):
        labels = labels.squeeze(1)  # TODO maybe remove squeeze
        pred = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return pred


def tokenize_fn(examples):
    tokenized_examples = tokenizer(
        examples["input_ids"], truncation=True, max_length=max_length, padding="max_length")
    labels = examples['labels']
    tok_labels = tokenizer(
        labels, truncation=True, max_length=max_length, padding="max_length")['input_ids']
    tokenized_examples['labels'] = tok_labels
    return tokenized_examples





In [ ]:
# labeled_df = pd.read_pickle('/home/student/Desktop/Project/Data/chords_en.pkl').head(100)[['chords&lyrics']].rename({'chords&lyrics': 'input_ids'}, axis=1)
datasets = DatasetDict({'train': Dataset.from_pandas(labeled_df)})
del labeled_df
model_name = 't5-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
max_length = 512

tokenized_datasets = datasets.map(tokenize_fn, remove_columns=datasets["train"].column_names)
tokenized_datasets.set_format('torch')

model = FillingModel()
OUT_PATH = Path("results")
args = TrainingArguments(output_dir=OUT_PATH, overwrite_output_dir=True, per_device_train_batch_size=8,
                         save_strategy='no',
                         do_train=True, num_train_epochs=5, report_to='none')

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("device: ", device)
model.to(device)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets['train'],
)
trainer.train()
model.model.save_pretrained('/home/student/Desktop/Project/Data/Models/model_basic')

0ex [00:00, ?ex/s]

/anaconda/envs/py38_default/lib/python3.8/site-packages/transformers/models/auto/modeling_auto.py:742: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

/anaconda/envs/py38_default/lib/python3.8/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272172048/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0
***** Running training *****
  Num examples = 160052
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 100035


Step,Training Loss
500,1.773800
1000,1.535500
1500,1.469700


concat song name at the begining
concat genres before fill
increase input length

